# 04 - Pipeline Final para Submissão

**🎯 PROPÓSITO DESTE NOTEBOOK:**
Este notebook implementa o pipeline final para gerar as previsões de vendas para janeiro de 2023. Utiliza todos os dados de 2022 para treinar o modelo final e gera os arquivos de submissão nos formatos requeridos.

**🚀 ESTRATÉGIA OTIMIZADA:**
- **Treino com Dataset Completo**: Usa todos os dados de 2022 (sem divisão treino/validação)
- **Modelo Final**: LightGBM treinado com número ótimo de iterações
- **Predições Janeiro/2023**: Gera features para as 5 semanas de janeiro de 2023
- **Submissão**: Salva nos formatos CSV e Parquet na pasta correta

---

## Fluxo do Pipeline:
1. **Carregamento**: Dados processados com features de `02-Feature-Engineering-Dask.ipynb`
2. **Preparação**: Otimização de memória e uso do dataset completo de 2022
3. **Treinamento**: Modelo final LightGBM com dados completos
4. **Predição**: Geração de features para janeiro/2023 e previsões finais
5. **Submissão**: Arquivos finais em CSV e Parquet prontos para envio

In [ ]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Advanced ML
import lightgbm as lgb
from lightgbm.callback import early_stopping  # CORREÇÃO: Importar early_stopping
import xgboost as xgb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from datetime import datetime, timedelta
import gc

plt.style.use('default')
sns.set_palette("husl")

print('📚 Bibliotecas carregadas com sucesso!')
print('🎯 Iniciando fase de Modelagem e Treinamento')

## 1. Carregamento dos Dados Processados

In [ ]:
# Carregar dados com features processadas
print('📂 Carregando dados processados...')

# Verificar se os arquivos essenciais existem
import os
required_files = [
    '../data/dados_features_completo.parquet',  # Usar parquet (mais rápido)
    '../data/feature_engineering_metadata.pkl'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print('❌ Arquivos não encontrados:')
    for f in missing_files:
        print(f'   • {f}')
    print('\n🔄 Execute primeiro o notebook 02-Feature-Engineering-Dask.ipynb')
else:
    print('✅ Todos os arquivos necessários encontrados')
    
    # Carregar dados principais (usar parquet para velocidade)
    print('📊 Carregando dataset (parquet)...')
    dados = pd.read_parquet('../data/dados_features_completo.parquet')
    
    # Carregar metadados
    with open('../data/feature_engineering_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    
    print(f'\n📊 Dados carregados com sucesso:')
    print(f'   • Shape: {dados.shape}')
    print(f'   • Período: {dados["semana"].min()} até {dados["semana"].max()}')
    print(f'   • Features disponíveis: {len(dados.columns)}')
    print(f'   • Memória: {dados.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    print(f'   • Estratégia: {metadata.get("estrategia", "Grid Inteligente")}')
    
    print(f'\n🔍 Metadados do processamento:')
    for key, value in metadata.items():
        if key not in ['features_criadas', 'data_processamento']:  # Skip long items
            print(f'   • {key}: {value}')
    
    print(f'\n✅ Pronto para modelagem!')

## 2. Preparação dos Dados para ML

In [ ]:
# Definir variável target e features
target = 'quantidade'

# Features a excluir (não devem ser usadas para predição)
exclude_features = [
    'pdv_id', 'produto_id', 'semana',  # IDs e data
    'quantidade',  # Target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro
]

# Identificar features disponíveis
all_features = [col for col in dados.columns if col not in exclude_features]

print(f'🎯 Preparação dos dados:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')

# Verificar missing values nas features
missing_features = dados[all_features].isnull().sum()
missing_features = missing_features[missing_features > 0].sort_values(ascending=False)

if len(missing_features) > 0:
    print(f'\n⚠️ Features com valores missing:')
    for feature, count in missing_features.head(10).items():
        pct = (count / len(dados)) * 100
        print(f'   • {feature}: {count:,} ({pct:.1f}%)')
    
    print(f'\n🧠 Estratégia de Tratamento Inteligente:')
    print('   • distributor_id (categórica): NaN → -1 (venda direta)')
    print('   • Features numéricas: NaN → 0 (ausência = zero)')
    print('   • LightGBM aprenderá padrões específicos para valores -1/0')
else:
    print('\n✅ Nenhum valor missing nas features')

print(f'\n📋 Features finais para modelagem: {len(all_features)}')
print('💡 Missing values serão tratados como informação, não removidos')

In [ ]:
# PIPELINE FINAL: Otimização de Memória + Preparação dos Dados Completos
print('🧠 Preparando dados de treino com o ano de 2022 completo...')
print('🚀 Estratégia: Usar todos os dados disponíveis para o modelo final')

# PASSO 1: Inspecionar uso de memória atual
print(f'\n🔍 ANTES da otimização:')
memory_before = dados.memory_usage(deep=True).sum() / (1024**3)
print(f'💾 Memória total: {memory_before:.2f} GB')

# PASSO 2: Aplicar Downcasting Inteligente
print(f'\n🚀 Aplicando Downcasting...')

# Fazer uma cópia para otimização
dados_sorted = dados.copy()

# Otimizar colunas numéricas (inteiros e floats)
for col in dados_sorted.select_dtypes(include=[np.number]).columns:
    original_dtype = dados_sorted[col].dtype
    
    if dados_sorted[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='integer')
    else:  # Floats
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='float')
    
    new_dtype = dados_sorted[col].dtype
    if original_dtype != new_dtype:
        print(f'   • {col}: {original_dtype} → {new_dtype}')

# Otimizar colunas categóricas
for col in dados_sorted.select_dtypes(include=['object']).columns:
    if col not in ['semana']:  # Preservar datetime
        nunique = dados_sorted[col].nunique()
        total_rows = len(dados_sorted)
        if nunique / total_rows < 0.5:  # Se <50% valores únicos, usar category
            dados_sorted[col] = dados_sorted[col].astype('category')
            print(f'   • {col}: object → category')

print(f'✅ Downcasting concluído!')

# PASSO 3: Verificar resultado da otimização
memory_after = dados_sorted.memory_usage(deep=True).sum() / (1024**3)
memory_reduction = (memory_before - memory_after) / memory_before * 100
print(f'\n📊 DEPOIS da otimização:')
print(f'💾 Memória total: {memory_after:.2f} GB')
print(f'🎯 Redução: {memory_reduction:.1f}% ({memory_before-memory_after:.2f} GB economizados)')

# PASSO 4: Tratamento de missing values
print(f'\n🧠 Tratamento inteligente de missing values...')
all_features = [col for col in dados_sorted.columns if col not in ['pdv_id', 'produto_id', 'semana', 'quantidade', 'valor', 'num_transacoes']]

for col in all_features:
    missing_count = dados_sorted[col].isnull().sum()
    if missing_count > 0:
        if col == 'distributor_id':
            # Adicionar -1 ao "menu" de categorias primeiro
            if dados_sorted[col].dtype.name == 'category':
                if -1 not in dados_sorted[col].cat.categories:
                    dados_sorted[col] = dados_sorted[col].cat.add_categories([-1])
            
            dados_sorted[col] = dados_sorted[col].fillna(-1)
            print(f'   • {col}: {missing_count:,} NaN → -1 (venda direta)')
            
        elif dados_sorted[col].dtype.kind in ['i', 'u', 'f']:
            dados_sorted[col] = dados_sorted[col].fillna(0)
            print(f'   • {col}: {missing_count:,} NaN → 0 (ausência)')

# PASSO 5: Preparar dados COMPLETOS para modelagem (SEM divisão treino/validação)
print(f'\n🎯 Preparando dados completos para o modelo final...')

# Usar todos os dados de 2022 para o treino final
X_full = dados_sorted[all_features]
y_full = dados_sorted[target]

print(f'✅ Dados de treino preparados:')
print(f'   • X_full shape: {X_full.shape}')
print(f'   • y_full shape: {y_full.shape}')
print(f'   • Período: {dados_sorted["semana"].min()} até {dados_sorted["semana"].max()}')
print(f'   • Memória X_full: {X_full.memory_usage(deep=True).sum() / (1024**2):.1f} MB')

# Limpeza de memória
import gc
del dados
gc.collect()

print(f'\n🎉 SUCESSO! Dados preparados para treinamento final:')
print(f'   ✅ Dataset completo de 2022: {len(dados_sorted):,} registros')
print(f'   ✅ Features otimizadas: {len(all_features)}')
print(f'   ✅ Memória otimizada: {memory_reduction:.1f}% de redução')
print(f'   ✅ Pronto para treinar modelo final!')

In [ ]:
# TREINAMENTO DO MODELO FINAL COM PARÂMETROS OTIMIZADOS
print('🚀 Treinando o modelo LightGBM final com todos os dados de 2022...')
print('🎯 Usando parâmetros otimizados pelo Optuna!')
print('=' * 60)

# Carregar os melhores parâmetros do Optuna
print('\n📂 Carregando melhores parâmetros de: ../data/best_lgbm_params_optuna.pkl')
try:
    with open('../data/best_lgbm_params_optuna.pkl', 'rb') as f:
        best_params_optuna = pickle.load(f)
    print('✅ Parâmetros do Optuna carregados com sucesso!')

    # Configurar parâmetros finais combinando Optuna + configurações fixas
    lgb_params_final = best_params_optuna.copy()
    lgb_params_final.update({
        'objective': 'regression_l1',  # MAE - melhor para WMAPE
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'random_state': 42,
        'n_jobs': -1
    })

    print(f'\n📋 Configuração do modelo final (Optuna + fixos):')
    for param, value in lgb_params_final.items():
        print(f'   • {param}: {value}')

    # Usar early stopping com todos os dados (sem validação separada)
    train_full_lgb = lgb.Dataset(X_full, label=y_full)

    print(f'\n📊 Preparando dados para treinamento...')
    print(f'   • Dados shape: {X_full.shape}')
    print(f'   • Features: {len(all_features)}')
    print(f'   • Target: {target}')
    print(f'   • Estratégia: Usar num_boost_round otimizado + early stopping interno')

    # Treinar modelo com parâmetros otimizados
    print(f'\n🔄 Treinando modelo final com parâmetros do Optuna...')

    # Usar o n_estimators do Optuna como num_boost_round
    num_boost_rounds = lgb_params_final.pop('n_estimators', 1000)

    final_model = lgb.train(
        lgb_params_final,
        train_full_lgb,
        num_boost_round=num_boost_rounds
    )

    print(f'✅ Modelo final treinado com sucesso!')
    print(f'   • Iterações utilizadas: {final_model.num_trees()}')

except FileNotFoundError:
    print('❌ Arquivo best_lgbm_params_optuna.pkl não encontrado!')
    print('🔄 Usando parâmetros padrão como fallback...')

    # Fallback para parâmetros vanilla
    lgb_params_final = {
        'objective': 'regression_l1',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'random_state': 42,
        'n_jobs': -1,
        'learning_rate': 0.1,
        'num_leaves': 31,
        'max_depth': -1
    }

    train_full_lgb = lgb.Dataset(X_full, label=y_full)
    final_model = lgb.train(
        lgb_params_final,
        train_full_lgb,
        num_boost_round=200
    )

    print(f'✅ Modelo fallback treinado com sucesso!')

# Salvar o modelo
import os
os.makedirs('../data', exist_ok=True)

with open('../data/final_lightgbm_model_optuna.pkl', 'wb') as f:
    pickle.dump(final_model, f)

print('💾 Modelo salvo em: ../data/final_lightgbm_model_optuna.pkl')

# Informações do modelo
print(f'\n📈 Informações do modelo final otimizado:')
print(f'   • Número de árvores: {final_model.num_trees()}')
print(f'   • Features utilizadas: {len(all_features)}')
print(f'   • Dados de treino: {len(y_full):,} registros')
print(f'   • Período de treino: Todo o ano de 2022')
print(f'   • Otimização: Parâmetros do Optuna')
print(f'   • Pronto para predições!')

## 3. Geração de Features para Janeiro/2023

In [ ]:
# GERAÇÃO DE FEATURES PARA JANEIRO/2023
print('📅 Criando grid de dados para as 5 semanas de janeiro/2023...')
print('🎯 Estratégia: Criar template para todas as combinações PDV/Produto')

# PASSO 1: Obter todas as combinações únicas e suas informações do histórico
print('\n🔍 Extraindo informações únicas de PDV/Produto do histórico...')
# CORREÇÃO: Usar 'dados_sorted' como fonte única de verdade para informações estáticas
available_cols = [col for col in ['pdv_id', 'produto_id', 'uf', 'distributor_id', 'brand'] if col in dados_sorted.columns]
info_basica_df = dados_sorted[available_cols].drop_duplicates(subset=['pdv_id', 'produto_id'])
print(f'   • Total de combinações únicas com informações: {len(info_basica_df):,}')
print(f'   • Colunas disponíveis: {available_cols}')

# PASSO 2: Definir as 5 semanas de janeiro de 2023
print('\n📅 Definindo as 5 semanas de janeiro/2023...')
prediction_weeks = pd.to_datetime(['2023-01-03', '2023-01-10', '2023-01-17', '2023-01-24', '2023-01-31'])
print(f'   • Semanas: {[w.strftime("%Y-%m-%d") for w in prediction_weeks]}')

# PASSO 3: Criar o grid de teste (todas as combinações x todas as semanas)
print('\n🏗️ Construindo grid de teste...')
test_grid_list = []
for week in prediction_weeks:
    combo_week = info_basica_df.copy()
    combo_week['semana'] = week
    test_grid_list.append(combo_week)

test_grid = pd.concat(test_grid_list, ignore_index=True)
print(f'   • Grid shape: {test_grid.shape}')
print(f'   • Total de predições: {len(test_grid):,}')

# PASSO 4: Adicionar features temporais
print('\n🔧 Gerando features temporais...')
test_grid['ano'] = test_grid['semana'].dt.year
test_grid['mes'] = test_grid['semana'].dt.month
test_grid['semana_ano'] = test_grid['semana'].dt.isocalendar().week
test_grid['dia_ano'] = test_grid['semana'].dt.dayofyear

# Features temporais trigonométricas (se existirem no treino)
if 'mes_sin' in all_features:
    test_grid['mes_sin'] = np.sin(2 * np.pi * test_grid['mes']/12)
    test_grid['mes_cos'] = np.cos(2 * np.pi * test_grid['mes']/12)

print('   • Features temporais adicionadas')

# PASSO 5: Calcular lags e rolling windows (usando dados_sorted como histórico)
print('\n📊 Calculando features de lag e estatísticas históricas...')

# Para cada combinação PDV/Produto, calcular estatísticas do histórico
historical_stats = dados_sorted.groupby(['pdv_id', 'produto_id'])['quantidade'].agg([
    'mean', 'std', 'min', 'max', 'count'
]).reset_index()
historical_stats.columns = ['pdv_id', 'produto_id', 'hist_mean', 'hist_std', 'hist_min', 'hist_max', 'hist_count']

# Merge com estatísticas históricas
test_grid = test_grid.merge(historical_stats, on=['pdv_id', 'produto_id'], how='left')

# Para os lags, vamos usar as últimas semanas de 2022
print('   • Calculando lags das últimas semanas de 2022...')
last_week_2022 = dados_sorted[dados_sorted['semana'] == dados_sorted['semana'].max()]
last_week_stats = last_week_2022[['pdv_id', 'produto_id', 'quantidade']].rename(columns={'quantidade': 'quantidade_lag_1'})
test_grid = test_grid.merge(last_week_stats, on=['pdv_id', 'produto_id'], how='left')

# Calcular mais lags se existirem no treino
unique_weeks = sorted(dados_sorted['semana'].unique(), reverse=True)
for lag in range(2, min(5, len(unique_weeks) + 1)):  # até lag_4
    if f'quantidade_lag_{lag}' in all_features:
        week_data = dados_sorted[dados_sorted['semana'] == unique_weeks[lag-1]]
        lag_stats = week_data[['pdv_id', 'produto_id', 'quantidade']].rename(columns={'quantidade': f'quantidade_lag_{lag}'})
        test_grid = test_grid.merge(lag_stats, on=['pdv_id', 'produto_id'], how='left')

# Rolling features (se existirem no treino)
lag_cols = [f'quantidade_lag_{i}' for i in range(1, 5) if f'quantidade_lag_{i}' in test_grid.columns]
if len(lag_cols) > 1:
    test_grid['quantidade_media_4w'] = test_grid[lag_cols].mean(axis=1)
    test_grid['quantidade_std_4w'] = test_grid[lag_cols].std(axis=1)
    test_grid['quantidade_max_4w'] = test_grid[lag_cols].max(axis=1)
    test_grid['quantidade_min_4w'] = test_grid[lag_cols].min(axis=1)

print('   • Features de lag e rolling adicionadas')

# PASSO 6: Preencher valores ausentes e finalizar
print('\n🔧 Finalizando e preenchendo valores ausentes...')

# Identificar colunas numéricas para preencher com 0
numeric_cols = test_grid.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    test_grid[col] = test_grid[col].fillna(0)

# Tratar distributor_id especificamente
if 'distributor_id' in test_grid.columns:
    if test_grid['distributor_id'].dtype.name == 'category':
        if -1 not in test_grid['distributor_id'].cat.categories:
            test_grid['distributor_id'] = test_grid['distributor_id'].cat.add_categories([-1])
    test_grid['distributor_id'] = test_grid['distributor_id'].fillna(-1)

print(f'\n✅ Features para janeiro/2023 geradas com sucesso!')
print(f'   • Grid final: {test_grid.shape}')
print(f'   • Colunas: {len(test_grid.columns)}')
print(f'   • Pronto para predições!')

## 4. Predições e Geração dos Arquivos de Submissão

In [ ]:
# PREDIÇÕES FINAIS E GERAÇÃO DE SUBMISSÃO
print('🚀 Realizando predições para janeiro/2023...')
print('🎯 Usando modelo final treinado com todo o dataset de 2022')

# PASSO 1: Preparar dados de teste com as mesmas features do treino
print('\n🔧 Preparando dados de teste...')

# Identificar features que existem tanto no treino quanto no teste
available_test_features = [col for col in all_features if col in test_grid.columns]
missing_features = [col for col in all_features if col not in test_grid.columns]

print(f'   • Features disponíveis no teste: {len(available_test_features)}')
if missing_features:
    print(f'   ⚠️ Features ausentes no teste: {len(missing_features)}')
    print(f'     Criando com valores padrão: {missing_features[:5]}...')
    
    # Criar features ausentes com valores padrão (0 ou -1)
    for feature in missing_features:
        if 'distributor' in feature or 'id' in feature:
            test_grid[feature] = -1
        else:
            test_grid[feature] = 0

# Selecionar apenas as features que foram usadas no treinamento
X_test = test_grid[all_features]

print(f'   • X_test shape: {X_test.shape}')
print(f'   • Features utilizadas: {len(all_features)}')

# PASSO 2: Aplicar o mesmo tratamento de tipos de dados
print('\n🔧 Aplicando otimizações de tipos de dados...')
for col in X_test.select_dtypes(include=[np.number]).columns:
    if X_test[col].dtype.kind in ['i', 'u']:  # Inteiros
        X_test[col] = pd.to_numeric(X_test[col], downcast='integer')
    else:  # Floats
        X_test[col] = pd.to_numeric(X_test[col], downcast='float')

# PASSO 3: Gerar predições
print('\n🎯 Gerando predições...')
predictions = final_model.predict(X_test)

# Garantir que as previsões não sejam negativas e arredondar para inteiros
predictions = np.maximum(0, predictions).round().astype(int)

print(f'   • Predições geradas: {len(predictions):,}')
print(f'   • Range: {predictions.min()} - {predictions.max()}')
print(f'   • Média: {predictions.mean():.2f}')
print(f'   • Zeros: {(predictions == 0).sum():,} ({(predictions == 0).mean()*100:.1f}%)')

# PASSO 4: Criar o DataFrame de submissão
print('\n📋 Criando DataFrame de submissão...')

# Preparar dados para submissão
submission_df = test_grid[['semana', 'pdv_id', 'produto_id']].copy()
submission_df['quantidade'] = predictions

# Converter semana para formato numérico (semana 1-5 de janeiro)
submission_df['semana_num'] = submission_df['semana'].dt.isocalendar().week
# Como são as semanas 1-5 de janeiro de 2023, ajustar numeração
primeira_semana = submission_df['semana_num'].min()
submission_df['semana'] = submission_df['semana_num'] - primeira_semana + 1

# Renomear colunas para o padrão da submissão
submission_df = submission_df.rename(columns={
    'semana': 'semana',
    'pdv_id': 'pdv', 
    'produto_id': 'produto'
})

# Selecionar apenas as colunas finais
submission_df = submission_df[['semana', 'pdv', 'produto', 'quantidade']]

print(f'   • DataFrame de submissão criado: {submission_df.shape}')
print(f'   • Colunas: {list(submission_df.columns)}')
print(f'   • Semanas: {sorted(submission_df["semana"].unique())}')

print(f'\\n📊 Estatísticas da submissão:')
for semana in sorted(submission_df['semana'].unique()):
    week_data = submission_df[submission_df['semana'] == semana]['quantidade']
    print(f'   • Semana {semana}: {len(week_data):,} predições, média: {week_data.mean():.1f}')

print(f'\\n✅ Dados de submissão preparados!')

In [ ]:
# SALVAMENTO DOS ARQUIVOS DE SUBMISSÃO FINAL OPTUNA
print('💾 Salvando arquivos de submissão FINAL OPTUNA...')
print('🎯 Formatos: CSV (separador ;) e Parquet')
print('🚀 Integração: Remover zeros automaticamente (como no notebook 05)')

# PASSO 1: Criar pasta de submissão
output_dir = '../submissions'
os.makedirs(output_dir, exist_ok=True)
print(f'   • Pasta criada: {output_dir}')

# PASSO 2: Validar DataFrame antes de salvar
print(f'\n🔍 Validação final do DataFrame:')
print(f'   • Shape: {submission_df.shape}')
print(f'   • Colunas: {list(submission_df.columns)}')
print(f'   • Tipos: {submission_df.dtypes.to_dict()}')
print(f'   • Missing values: {submission_df.isnull().sum().sum()}')
print(f'   • Valores negativos: {(submission_df["quantidade"] < 0).sum()}')
print(f'   • Valores zero: {(submission_df["quantidade"] == 0).sum():,} ({(submission_df["quantidade"] == 0).mean()*100:.1f}%)')

# Verificar se há problemas
if submission_df.isnull().sum().sum() > 0:
    print('   ⚠️ Há valores missing - preenchendo com 0')
    submission_df = submission_df.fillna(0)

if (submission_df["quantidade"] < 0).sum() > 0:
    print('   ⚠️ Há valores negativos - convertendo para 0')
    submission_df["quantidade"] = submission_df["quantidade"].clip(lower=0)

# PASSO 3: 🎯 FILTRAR ZEROS (INTEGRAÇÃO DO NOTEBOOK 05)
print(f'\n🔧 PASSO CRÍTICO: Filtrando para manter apenas previsões com quantidade > 0...')
submission_df_full = submission_df.copy()  # Backup da submissão completa
submission_final = submission_df[submission_df['quantidade'] > 0].copy()

linhas_removidas = len(submission_df_full) - len(submission_final)
print(f'   • Linhas removidas (zeros): {linhas_removidas:,}')
print(f'   • Linhas mantidas: {len(submission_final):,}')
print(f'   • Redução: {(linhas_removidas/len(submission_df_full))*100:.1f}%')

# PASSO 4: Salvar arquivo COMPLETO (com zeros) - para backup
print(f'\n💾 Salvando arquivo COMPLETO (backup):')
csv_complete_path = os.path.join(output_dir, 'submission_complete_with_zeros.csv')
parquet_complete_path = os.path.join(output_dir, 'submission_complete_with_zeros.parquet')

submission_df_full.to_csv(csv_complete_path, sep=';', index=False, encoding='utf-8')
submission_df_full.to_parquet(parquet_complete_path, index=False)
print(f'   • CSV completo: {csv_complete_path}')
print(f'   • Parquet completo: {parquet_complete_path}')

# PASSO 5: Salvar arquivo FINAL OPTUNA (sem zeros) - para submissão
print(f'\n🏆 Salvando arquivo FINAL OPTUNA (pronto para submissão):')
csv_final_path = os.path.join(output_dir, 'submission_final_optuna.csv')
parquet_final_path = os.path.join(output_dir, 'submission_final_optuna.parquet')

submission_final.to_csv(csv_final_path, sep=';', index=False, encoding='utf-8')
submission_final.to_parquet(parquet_final_path, index=False)

print(f'   • CSV FINAL: {csv_final_path}')
print(f'   • Parquet FINAL: {parquet_final_path}')

# PASSO 6: Verificar tamanhos dos arquivos
csv_final_size = os.path.getsize(csv_final_path) / (1024**2)  # MB
parquet_final_size = os.path.getsize(parquet_final_path) / (1024**2)  # MB

print(f'\n📊 Estatísticas dos arquivos FINAIS:')
print(f'   • CSV FINAL: {csv_final_size:.2f} MB')
print(f'   • Parquet FINAL: {parquet_final_size:.2f} MB')

# PASSO 7: Estatísticas finais da submissão
print(f'\n📈 Estatísticas da SUBMISSÃO FINAL OPTUNA:')
for semana in sorted(submission_final['semana'].unique()):
    week_data = submission_final[submission_final['semana'] == semana]
    print(f'   • Semana {semana}: {len(week_data):,} predições, média: {week_data["quantidade"].mean():.1f}')

print(f'\n📊 Distribuição de quantidade na submissão final:')
print(f'   • Mín: {submission_final["quantidade"].min()}')
print(f'   • Máx: {submission_final["quantidade"].max()}')
print(f'   • Média: {submission_final["quantidade"].mean():.2f}')
print(f'   • Mediana: {submission_final["quantidade"].median():.1f}')
print(f'   • Total de predições: {submission_final["quantidade"].sum():,}')

# PASSO 8: Visualizar preview da submissão final
print(f'\n👀 Preview da SUBMISSÃO FINAL OPTUNA (primeiras 10 linhas):')
print(submission_final.head(10).to_string(index=False))

print(f'\n🎉 PROCESSO FINALIZADO - SUBMISSION FINAL OPTUNA PRONTA!')
print(f'=' * 70)
print(f'✅ Modelo LightGBM Optuna treinado com {len(y_full):,} registros de 2022')
print(f'✅ Parâmetros otimizados pelo Optuna aplicados')
print(f'✅ Predições filtradas: {len(submission_final):,} (sem zeros)')
print(f'✅ Arquivo pronto: submission_final_optuna.csv')
print(f'✅ Formato: CSV com separador ";" conforme regulamento')
print(f'')
print(f'🚀 ARQUIVO PRONTO PARA SUBMISSÃO:')
print(f'   📂 {csv_final_path}')
print(f'   📏 {csv_final_size:.2f} MB')
print(f'   📊 {len(submission_final):,} linhas')
print(f'')
print(f'🏆 BOA SORTE NO HACKATHON!')